This is the fifth session on basic usage of Python 3, which is on pandas usage.

## Course Structure

### Session 4 [2]

**Even More with pandas**
* Sorting and ranking

* Unique Values, Value Counts

* Data Wrangling
    - Clean
    - Transform,
    - Merge
    - Reshape
    
* Data Transformation
    - Removing duplicates
    - Using a Function or Mapping
    - Replacing Values
    - Discretization and Binning
    - Detecting and Filtering Outliers
    - Data Aggregation and Group Operations
    - Grouping with Dicts and Series


## Discretization and Binning

In [2]:
import pandas as pd
import numpy as np

<font color = 'blue'> Continuous data is often discretized or otherwised separated into “bins” for analysis. <br>
Suppose you have data about a group of people in a study, and you want to group them
into discrete age buckets:

In [11]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

In [14]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

<font color = 'blue'> The object pandas returns is a special Categorical object. You can treat it like an array
of strings indicating the bin name; internally it contains a levels array indicating the
distinct category names along with a labeling for the ages data in the labels attribute:

<font color = 'blue'>
You can also pass your own bin names by passing a list or array to the labels option:: <fobt color = 'blue'>

In [18]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

<font color = 'blue'> If you pass cut a integer number of bins instead of explicit bin edges, it will compute
equal-length bins based on the minimum and maximum values in the data.

In [19]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.1, 0.33], (0.55, 0.77], (0.55, 0.77], (0.77, 1.0], (0.1, 0.33], ..., (0.77, 1.0], (0.77, 1.0], (0.55, 0.77], (0.33, 0.55], (0.77, 1.0]]
Length: 20
Categories (4, interval[float64]): [(0.1, 0.33] < (0.33, 0.55] < (0.55, 0.77] < (0.77, 1.0]]

<font color = 'blue'>   A closely related function, qcut, bins the data based on sample quantiles. Depending
on the distribution of the data, using cut will not usually result in each bin having the
same number of data points:

In [20]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

## Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider
a DataFrame with some normally distributed data:

In [21]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
col = data[3]

In [22]:
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

To select all rows having a value exceeding 3 or -3, you can use the any method on a
boolean DataFrame:

In [23]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


## Data Aggregation and Group Operations

<font color = 'blue'>  One reason for the popularity of relational databases and SQL is the ease with which data can be joined, filtered, transformed,
and aggregated. However, query languages like SQL are rather limited in the
kinds of group operations that can be performed. As you will see, with the expressiveness
and power of Python and pandas, we can perform much more complex grouped
operations by utilizing any function that accepts a pandas object or NumPy array.

In [25]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})

In [12]:
df

,key1,key2,data1,data2
0,a,one,-1.420671,1.515486
1,a,two,0.904438,-0.647692
2,b,one,0.426093,-0.473329
3,b,two,-0.994206,0.201410
4,a,one,0.536225,1.808997


<font color = 'blue'>  Suppose you wanted to compute the mean of the data1 column using the groups labels
from key1

In [122]:
grouped = df['data1'].groupby(df['key1'])

This grouped variable is now a GroupBy object. It has not actually computed anything
yet except for some intermediate data about the group key df['key1'].

In [124]:
grouped.mean()

,rkey,data2
0,a,0
1,b,1
2,d,2


If instead we had passed multiple arrays as a list, we get something different:

In [15]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.442223
      two     0.904438
b     one     0.426093
      two    -0.994206
Name: data1, dtype: float64

In [17]:
df

,key1,key2,data1,data2
0,a,one,-1.420671,1.515486
1,a,two,0.904438,-0.647692
2,b,one,0.426093,-0.473329
3,b,two,-0.994206,0.201410
4,a,one,0.536225,1.808997


In [16]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.006664,0.892264
b,-0.284056,-0.135960


In [18]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.442223  1.662241
     two   0.904438 -0.647692
b    one   0.426093 -0.473329
     two  -0.994206  0.201410

In [19]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.662241
     two  -0.647692
b    one  -0.473329
     two   0.201410

## Grouping with Dicts and Series

In [21]:
people = pd.DataFrame(np.random.randn(5, 5),
    columns=['a', 'b', 'c', 'd', 'e'],
    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people

,a,b,c,d,e
Joe,-0.913743,-2.795339,-3.949305,-0.508680,0.018986
Steve,0.400246,1.284082,-0.712018,-1.132599,-0.882907
Wes,2.422803,-1.070286,-1.114160,0.589662,0.411347
Jim,-1.185749,-0.044376,0.615255,0.101992,0.616957
Travis,0.984461,0.612704,-1.976774,1.107612,-0.031442


In [25]:
people.iloc[2:3, 1:3] = np.nan
people

,a,b,c,d,e
Joe,-0.913743,-2.795339,-3.949305,-0.508680,0.018986
Steve,0.400246,1.284082,-0.712018,-1.132599,-0.882907
Wes,2.422803,NaN,NaN,0.589662,0.411347
Jim,-1.185749,-0.044376,0.615255,0.101992,0.616957
Travis,0.984461,0.612704,-1.976774,1.107612,-0.031442


In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

suppose I have a group correspondence for the columns and want to sum together
the columns by group:

In [29]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-4.457985,-3.690096
Steve,-1.844617,0.801420
Wes,0.589662,2.834150
Jim,0.717247,-0.613167
Travis,-0.869161,1.565722


## Data Aggregation

In [27]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.841616
b   -0.019206
Name: data1, dtype: float64

<font color = 'blue'>  To use your own aggregation functions, pass any function that aggregates an array to
the aggregate or agg method

In [28]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [29]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.147939,1.651730
b,0.656928,0.001871


<font color = 'blue'> Some methods like describe also work, even though they are not
aggregations, strictly speaking:

In [30]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.080463  1.107972 -0.997174 -0.696078 -0.394982  0.377892   
b      2.0 -0.281977  0.464518 -0.610441 -0.446209 -0.281977 -0.117746   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.150765   3.0  0.253098  0.852022 -0.451814 -0.220310  0.011194   
b     0.046486   2.0 -0.154450  0.001323 -0.155385 -0.154917 -0.154450   

                          
           75%       max  
key1                      
a     0.605554  1.199915  
b    -0.153982 -0.153514

### Advanced aggregation methods:

In [31]:
tips = pd.read_csv(r'C:\Users\singhn25\Downloads\Python_Sessions\tips.csv')

In [32]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [33]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [41]:
grouped_pct = tips.groupby(['sex', 'smoker'])['tip_pct'].agg('mean')

In [42]:
grouped_pct

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

### Column-wise and Multiple Function Application

In [43]:
grouped_pct = tips.groupby(['sex', 'smoker'])['tip_pct'].agg(['mean', 'std', peak_to_peak])
grouped_pct

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [45]:
grouped_pct = tips.groupby(['sex', 'smoker'])['tip_pct'].agg([('my_mean' , 'mean'), ('my_std', 'std'), ('max-min', peak_to_peak)])
grouped_pct

my_mean    my_std   max-min
sex    smoker                              
Female No      0.156921  0.036421  0.195876
       Yes     0.182150  0.071595  0.360233
Male   No      0.160669  0.041849  0.220186
       Yes     0.152771  0.090588  0.674707

### Filling Missing Values with Group-specific Values

In [59]:
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.DataFrame({'group_key':group_key, 'values': np.random.randn(8)}, index=states)
data.loc[['Vermont', 'Nevada', 'Idaho'], 'values'] = np.nan

In [60]:
data

,group_key,values
Ohio,East,-1.338431
New York,East,1.880710
Vermont,East,NaN
Florida,East,-0.348691
Oregon,West,-0.553122
Nevada,West,NaN
California,West,1.544168
Idaho,West,NaN


In [61]:
data.groupby(group_key).mean()

,values
East,0.064529
West,0.495523


In [62]:
data.groupby(group_key).apply(lambda g: g.fillna(g.mean()))

group_key    values
East Ohio            East -1.338431
     New York        East  1.880710
     Vermont         East  0.064529
     Florida         East -0.348691
West Oregon          West -0.553122
     Nevada          West  0.495523
     California      West  1.544168
     Idaho           West  0.495523